Normalization

In [ ]:
data=[-8.42182038, -6.7079959,   8.69898508, -0.98348064, -0.56839347,  4.73965983, 1.74041161,  1.31899395,  3.40522183,  3.08001056, -7.76980902,  0.41988858]

scaled = [-1 + 2 * (x - min(data)) / (max(data) - min(data)) for x in data]
print(scaled)
# [-1.0, 1.0, 0.3599364886029235]

# using scikit-learn's MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1, 1))
scaled = scaler.fit_transform([[x] for x in data])
print(scaled)

In [ ]:
!nvidia-smi

Sun Mar 12 07:15:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    29W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.5 MB/s eta 0:00:00
time: 347 µs (started: 2023-03-12 07:15:37 +00:00)


In [ ]:
!pip install nufhe[pycuda]
!pip install pycuda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 KB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.9/191.9 KB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 41.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 KB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 11.3 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2022.2.2-cp39-cp39-linux_x86_64.whl size=661964 sha256=f341ad79554a5d9ff787d0f26ca640d9daf2d9f01128fc92df7e3efcf56cfc1a
  Stored in directory: /root/.cache/pip/wheels/82/09/27/74d8e31ed19c530166e0d263aabe1ea57465e255615bda8fc0
  Created w

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 22.2 s (started: 2023-03-12 07:18:28 +00:00)


In [3]:
import time
import random
#import pycuda.autoinit
import nufhe
import numpy as np
import numpy as np
from datetime import datetime
def cba4(r,a,b,carry):
    temp= vm.gate_and(a, b) #ab
    temp1= vm.gate_and( b,carry ) #bc
    temp2= vm.gate_and(a, carry) # ac
    temp3= vm.gate_or(temp, temp1) #ab+ac
    r[1]=vm.gate_or(temp3,temp2) # ab+abc+ac
    r[0]=vm.gate_not(r[1]) #(ab+bc+ca)'
    return r
def cba1(r,a,b,carry):
    temp= vm.gate_and(b,carry) #temp=bc
    r[1]=vm.gate_or(temp, a)   #r1=bc+a 
    r[0]=vm.gate_not(r[1])     #r0=(bc+a)'
    return r

def threetotwo(r,p0,p1,p2):
    r[0] = vm.gate_or(p0, p1)
    temp=vm.gate_and(p0, p1)
    r[1]=vm.gate_or(temp, p2)
    return r
def twotoone(r, p0,p1 ):
    r[0] = vm.gate_or(p0, p1)
    return r
def is_equal(a,b,nBits):
    #print("type of a",type(a))
    #print("type of b",type(b))      
    t2= ctx.encrypt(secret_key, [False])
    #print("length of list",len(a))
    for i in range(0,nBits):
        t1=vm.gate_xor(a[i], b[i])
        t3=vm.gate_or(t2, t1)
        t2=vm.gate_copy(t3)
        equal=vm.gate_not(t3)

    return equal
def make_neg(n,nbits):
    list1=[int(i) for i in range(0,len(n)) ]
    listp=[]
    #one=vm.empty_ciphertext((1,))
    #zero=[vm.empty_ciphertext((1,))]
    #one= [ctx.encrypt(secret_key, [True])]
    #zero= [ctx.encrypt(secret_key, [False])]
    #t2= [ctx.encrypt(secret_key, [False])]
    #one=  [ctx.encrypt(secret_key, [False]) for _ in range(nbits)]
    #zero=  [ctx.encrypt(secret_key, [False) for _ in range(nbits)]
    #print("type of one is",type(one))
    #print("typeof zero is",type(zero))
    #print("tupe of one is",one)
    #[vm.empty_ciphertext((1,))]
    '''for i in range(0,len(n)):
        #temp=vm.gate_copy(n[i])
        temp=vm.gate_mux(n[i],zero[0],one[0])
        n[i]=temp[:]'''
            
    
    #print(n)
    one= ctx.encrypt(secret_key, [True])
    onep=  [ctx.encrypt(secret_key, [False]) for _ in range(nbits)]
    onep[0]=one 
    testone= [[vm.empty_ciphertext((1,))] for i in range(nbits)]
    testone=onep[:]
    '''temp1=n[:]
    temp1.reverse()
    result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(nBits)]
    pa=listToString(result_bits)
    print("n before",twos_comp_val(int(pa,2),len(pa)))'''
    n=addNumbers(n,testone,nbits)
    '''temp1=n[:]
    temp1.reverse()
    result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(nBits)]
    pa=listToString(result_bits)
    print("n after",twos_comp_val(int(pa,2),len(pa)))'''
    return n
def mul(ciX,ciY,zero_n):
    
    msb_temp= [vm.empty_ciphertext((1,))]
    control=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    c_s=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    c_s1=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    msb_temp=vm.gate_xor(ciX[size-1],ciY[size-1])
    t=size-1
    for i in range(size):
            control[i]=ciX[t]
            c_s1[i]=ciY[t]
    enc_temp=zero_n
    mul_temp=ciX
    zero=enc_temp
    one_ciX=vm.gate_not(ciX)
    result=ctx.decrypt(secret_key, one_ciX)
    one_ciY=vm.gate_not(ciY)
    comp_ciX=add(one_ciX,one,zero_n)
    comp_ciY=add(one_ciY,one,zero_n)
    
    #print("control bit:",ctx.decrypt(secret_key, control))
    ciX=mux(control,comp_ciX,ciX)
    ciY=mux(c_s1,comp_ciY,ciY)
    #print("cix:",ctx.decrypt(secret_key, ciX))
    for j in range(0,size):
        #print("=================iteration==========",j)
        
        
        #print("cix:",ctx.decrypt(secret_key, ciX))
        for i in range(size):
            c_s[i]=ciY[j]
        add_res=mux(c_s,ciX,zero)
        
        enc_theta=add(enc_temp,add_res,zero_n)
        enc_temp=enc_theta
        
        '''print("enc_teta:",ctx.decrypt(secret_key, enc_theta))
        result=ctx.decrypt(secret_key, enc_theta)
        val=listToString(result)
        print("enc_theta ",two2dec(val[::-1]) )
        
        
        print("control bit:",ctx.decrypt(secret_key, c_s[0]))'''
        
        ciX.roll(1, axis=0)
        #ciX.roll(1, axis=0)
    for i in range(size):
        c_s[i]=msb_temp    
    one_presult=vm.gate_not(enc_theta)
    comp_presult=add(one_presult,one,zero_n)
    
    ciX=vm.gate_mux(c_s,comp_presult,enc_theta)
    enc_theta[size-1]=msb_temp 
    #return enc_theta
    return ciX



def nonRestoringDivision(m, q,nBits,zero_n ): 
    A=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    temp=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    mux1=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    mux2=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    neg2=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    q_c=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    mcopy=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    onep=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    fullzero=ctx.encrypt(secret_key, [[False] for _ in range(size)])
    one= ctx.encrypt(secret_key, [True])
    zero= ctx.encrypt(secret_key, [False])
    signbit= ctx.encrypt(secret_key, [[False] for _ in range(size)])
    #tw= ctx.encrypt(secret_key, [False])
    '''for i in range(len(q)):
        q_c=nufhe.LweSampleArray.copy(q[i])'''
    onep[0]=vm.gate_copy(one)  
    c_zero= ctx.encrypt(secret_key, [[False] for _ in range(size)])
    testone= ctx.encrypt(secret_key, [[False] for _ in range(size)])
    ones= ctx.encrypt(secret_key, [[False] for _ in range(size)])
    
    c_zero=zero
    #A[i]=vm.gate_copy(zero)
    testone=onep
    q_c=q
    #A[i]=zero
    A=fullzero
    mcopy=m
    temp=c_zero
    mux1=fullzero
    equal_res= ctx.encrypt(secret_key, [False])
    
    #temp=m
    ones=vm.gate_not(m)
    '''result=ctx.decrypt(secret_key,ones)
    val=listToString(result)
    print(val)
    print("ones comlplement ",two2dec(val[::-1]) )'''
    neg2=add(ones,onep,zero_n)
    '''result=ctx.decrypt(secret_key,onep)
    val=listToString(result)
    print(val)
    print("onep is",two2dec(val[::-1]) )'''
    '''result=ctx.decrypt(secret_key,neg2)
    val=listToString(result)
    print(val)
    print("neg2 value ",two2dec(val[::-1]))'''
    
    
    count=nBits
    #print("********* loop started **************")
    
    while (True): 
        #print("===loop==",count)
        for i in range(nBits):
            signbit[i]=A[nBits-1]
        
        '''signplain=[]
        signplain.append([ctx.decrypt(secret_key,signbit)])
        print("signbit is",signplain)'''
        for i in range(nBits-1,0,-1):
            A[i] =A[i-1]    #a=a>>1
        A[0]=q_c[nBits-1]             #a[0]=q[msb]  
        for i in range(nBits-1,0,-1):
            q_c[i] =q_c[i-1]              #q=q>>1 
        '''result=ctx.decrypt(secret_key,A)
        val=listToString(result)
        print(val)
        print("A after left shitf ",two2dec(val[::-1]) )'''
        #aminusm=subtract(A,m,nBits)
        #aplusm=addNumbers(A,m,nBits)
        
        mux1=mux(signbit,mcopy,neg2)   #mux1= mcopy,neg2
        '''result=ctx.decrypt(secret_key,mux1)
        val=listToString(result)
        print(val)
        print("mux1 ",two2dec(val[::-1]) )'''
        result=ctx.decrypt(secret_key,A)
        '''val=listToString(result)
        print(val)
        print("a before add ",two2dec(val[::-1]) )
        result=ctx.decrypt(secret_key,mcopy)
        val=listToString(result)
        print("m before add ",two2dec(val[::-1]) )'''
            
        temp1=add(A,mux1,zero_n)     #a=a+m or a-m
        
        A=temp1
        '''result=ctx.decrypt(secret_key,A)
        val=listToString(result)
        print("a is",val)
        print("A after add or subration",two2dec(val[::-1]) )'''
        
        #A=mux1[:]
        #equal_res=is_equal(A,c_zero,nBits)
        signbit[0]=A[nBits-1]
        '''signplain1=[]
        signplain1.append([ctx.decrypt(secret_key,signbit[0])])
        print("@@@ signbit beforeupdate of q_C",signplain1)'''
        tw1=mux(signbit[0],zero,one)      
        '''signplain2=[]
        signplain2.append([ctx.decrypt(secret_key,tw1)])
        print("@@@ tw1 beforeupdate of q_C",signplain2)'''
        q_c[0]=tw1   #Q[0]=select(A<zero,zero,one);
        #q_c.insert(0,tw1)
        #temp2 = [[vm.empty_ciphertext((1,))] for i in range(nBits)]
        
        '''result=ctx.decrypt(secret_key,A)
        val=listToString(result)
        print("a is ",val)
        print("count is" ,count,"A inside loop",two2dec(val[::-1]))'''
        
        temp2=q_c
        '''result=ctx.decrypt(secret_key,q_c)
        val=listToString(result)
        print("q_c is",val)
        print("count is" ,count,"q_c inside loop",two2dec(val[::-1]) )'''
        
        count -= 1
        if count==0:
            break
    #aminuszero=subtract(A,c_zero,nBits)
    for i in range(nBits):
        signbit[i]=A[nBits-1]
    aminuszero=A
    aplusm =add(A,m,zero_n)
    
    A=mux(signbit,aplusm,aminuszero)    #A=select(A<zero,A+M,A);
    return q_c
    #return A,q_c 


def fixSizeBoolList(decimal,size):
    x = [int(x) for x in bin(decimal)[2:]]
    x = list(map(bool, x))
    x = [False]*(size - len(x)) + x
    pow2 = []
    for i in range(size):
      pow2.append([x[i]])
    pow2.reverse()
    return pow2
def boolListToInt(bitlists):
    out = 0
    for bit in bitlists:
        out = (out << 1) | bit
    return out

def addBits(r, a, b, carry):
    # Xor(t1[0], a, carry[0])
    t1 = vm.gate_xor(a, b)
    # Xor(t2[0], b, carry[0])
    # Xor(r[0], a, t2[0])
    r[0] = vm.gate_xor(t1, carry)
    # And(t1[0], t1[0], t2[0])
    t2 = vm.gate_and(a, carry)
    t3 = vm.gate_and(b, carry)
    t4=vm.gate_and(a,b)
    t5= vm.gate_or(t2,t3)
    # Xor(r[1], carry[0], t1[0])
    r[1] = vm.gate_or(t5, t4)

    return r
 

def addNumbers(ctA, ctB, nBits):
    ctRes = [[vm.empty_ciphertext((1,))] for i in range(nBits)]
    # carry = vm.empty_ciphertext((1,))
    bitResult = [[vm.empty_ciphertext((1,))] for i in range(2)]
    ctRes[0] = vm.gate_xor(ctA[0], ctB[0])
    # Xor(ctRes[0], ctA[0], ctB[0])
    carry = vm.gate_and(ctA[0], ctB[0])
    # And(carry[0], ctA[0], ctB[0])
    for i in range(1,nBits ):
      if i>0:
        bitResult = addBits(bitResult, ctA[i], ctB[i], carry)
        # Copy(ctRes[i], bitResult[0]);
        ctRes[i] = nufhe.LweSampleArray.copy(bitResult[0])
        # Copy(carry[0], bitResult[1])
        carry = nufhe.LweSampleArray.copy(bitResult[1])
      else:
        bitResult = twotoone(bitResult, ctA[i], ctB[i])
        #bitResult = threetotwo(bitResult, ctA[i], ctB[i],carry)
        #bitResult=cba4(bitResult, ctA[i], ctB[i],carry)
        #Copy(ctRes[i], bitResult[0]);
        ctRes[i] = nufhe.LweSampleArray.copy(bitResult[0])
        #carry = nufhe.LweSampleArray.copy(bitResult[1])
        
    return ctRes


def mulNumbers(ctA, ctB, secret_key, input_bits, output_bits):
    result = [ctx.encrypt(secret_key, [False]) for _ in
              range(output_bits)]
    temp = [ctx.encrypt(secret_key, [False]) for _ in
              range(output_bits)]          
    # [[vm.empty_ciphertext((1,))] for _ in range(output_bits)]
    # andRes = [[vm.empty_ciphertext((1,))] for _ in range(input_bits)]
    
    for i in range(input_bits):
        andResLeft = [ctx.encrypt(secret_key, [False]) for _ in
                      range(output_bits)]
        #temp=mux(temp,ctA,ctB[i],size)             
        for j in range(input_bits):
            andResLeft[j + i] = vm.gate_and(ctA[j], ctB[i])

            # andResLeft[j + i] = nufhe.LweSampleArray.copy(andRes[j])
        result = addNumbers(andResLeft, result, output_bits)
        #result_bits = [ctx.decrypt(secret_key, result[i]) for i in range(size * 2)]
        #print(" nuFHE multiplication intermdiate number is : ",i,boolListToInt(result_bits))

    return result[:size]
def Convert_list(string):
    list1=[]
    list1[:0]=string
    print(list1)
    list1=[int(i)for i in list1 ]
    listb=[]
    for i in list1:
        if i==0:
            listb.append([False])
        else:
            listb.append([True])    
    
    #print(listb)
    return listb

def twos_complement(n,nbits):
    a=f"{n & ((1 << nbits) - 1):0{nbits}b}" 
    #print(type(a))
    a=Convert_list(a)
    a.reverse()
    return a
def listToString(s): 
    # initialize an empty string
    list1=[int(i)for i in s ]
    listp=[]
    for i in list1:
        if i==False:
            listp.append('0')
        else:
            listp.append('1')    
    
    #print(listp)
    str1 = "" 
    # traverse in the string  
    s=['delim'.join([str(elem) for elem in sublist]) for sublist in listp]
    #print(s)
    for ele in s: 
        str1 += ele  
    # return string 
    #print(str1)
    return str1
def twos_comp_val(val,bits):
    """compute the 2's complement of int value val"""
    #val=listToString(val)
    

    if (val & (1 << bits - 1)) != 0: # if sign bit is set e.g., 8bit: 128-255
        val = val - (1 << bits)        # compute negative value
    return val
def compare_bit(  a, b,  lsb_carry,  tmp): 
  result= ctx.encrypt(secret_key, [False])
  tmp=vm.gate_xnor(a, b)
  result=vm.gate_mux(tmp,lsb_carry, a)
  return result
def minimum(  a,  b,  nb_bits): 
    tmps1= ctx.encrypt(secret_key, [False])
    tmps2= ctx.encrypt(secret_key, [True])
    #initialize the carry to 0
    #run the elementary comparator gate n times
    for i in range(nb_bits):
      tmps1= compare_bit(a[i],b[i],tmps1,tmps2)
    #tmps[0] is the result of the comparaison: 0 if a is larger, 1 if b is larger
    #select the max and copy it to the result
    return tmps1
def predict(ctA,onep,onen):
    c_s=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    for i in range(size):
        c_s[i]=ctA[size-1]
        
    res= vm.gate_mux(c_s,onen,onep)
    return res
def add(ciX, ciY,temp):
# fixed iteration since
    
    for i in range(size):
        #print("=================iteration==========",i)
        a = vm.gate_and(ciX, ciY)
        b = vm.gate_xor(ciX, ciY)
        '''result=ctx.decrypt(secret_key,a)
        print(result)
        print("carry  is",boolListToInt(result))'''
        aShiftTemp = a
        # using roll as a shift bit
        aShiftTemp.roll(1, axis=0)
        #aShiftTemp=np.roll(aShiftTemp,-1, axis=0)
        ciX = aShiftTemp
        ciX[0]=temp[0]
        ciY = b
        '''result=ctx.decrypt(secret_key,ciX)
        print(result)
        print("carry after shift",boolListToInt(result))
        result=ctx.decrypt(secret_key,ciY)
        print(result)    
        print("sum is",boolListToInt(result))'''
    #result=ctx.decrypt(secret_key,b)
    #print("result is",boolListToInt(result))
    return b
def mux(c_s,true,false) :
    temp1=vm.gate_and(c_s,true)
    #print("temp1:",ctx.decrypt(secret_key, temp1))
    temp2=vm.gate_and(vm.gate_not(c_s),false)
    #print("temp2:",ctx.decrypt(secret_key, temp2))
    mux_result=vm.gate_or(temp1,temp2)
    return mux_result
def two2dec(s):
    if s[0] == '1':
        return -1 * (int(''.join('1' if x == '0' else '0' for x in s), 2) + 1)
    else:
        return int(s, 2)
if __name__ == '__main__':

    size=20
    ctx = nufhe.Context()
    secret_key, cloud_key = ctx.make_key_pair()
    vm = ctx.make_virtual_machine(cloud_key)
    with open('/home/pradeep/Downloads/pc11.txt') as f:
        lines = []
        for line in f:
            lines.append([int(v) for v in line.split()])
    w_p=[-4, 6, -1, -1, 3, 1, 1, 2, 2, -4, 1, 6]
    zero= np.zeros((size,1), dtype=bool)
    A = fixSizeBoolList(7,size)
    B = fixSizeBoolList(9,size)
    s=[ [False] for i in range(size)]
    test_size = int(input("Please enter a string:\n"))
    theta = fixSizeBoolList(0,size)
    temp=fixSizeBoolList(0,size)
    Y = fixSizeBoolList(7,size)
    one=ctx.encrypt(secret_key, twos_complement(1,size))
    ciX=ctx.encrypt(secret_key, A)
    ciY=ctx.encrypt(secret_key, B)
    c_s=ctx.encrypt(secret_key, s)
    #onen=  [ctx.encrypt(secret_key, [True]) for _ in range(size)]        
    #onep=  [ctx.encrypt(secret_key, [False]) for _ in range(size)]
    print("==============")

    print("cix:",ctx.decrypt(secret_key, ciX))

    #enc_theta=ctx.encrypt(secret_key, theta)
    enc_temp=ctx.encrypt(secret_key, temp)
    enc_temp1=enc_temp
    #print("type of cx is",type(ciX[0]))
    #print("type is",type(enc_temp[0]))
    print("==============")
    enc_val=ctx.encrypt(secret_key, Y)
    abc=vm.gate_constant(zero)
    '''begin = time.time()
    #not_result=equal_check(ciX, ciY)
    mul_temp=ciX
    for j in range(0,size-1):
        print("=================iteration==========",j)
        ciX=ciX
        print("cix:",ctx.decrypt(secret_key, ciX))

        enc_theta=add(ciX,mul_temp)
        print("enc_teta:",ctx.decrypt(secret_key, enc_theta))
        for i in range(size):
            c_s[i]=ciY[j] 
        print("control bit:",ctx.decrypt(secret_key, c_s[0]))
        mux_result=mux(c_s,enc_theta,ciX)
        #mux_result=vm.gate_mux(c_s, enc_theta,ciX)
        print("mux result:",ctx.decrypt(secret_key, mux_result))
        ciX=mux_result
    end = time.time()'''

    n=twos_complement(-1,size)
    p=twos_complement(1,size)
    onen=ctx.encrypt(secret_key, n)
    onep=ctx.encrypt(secret_key, p)
    eten=ctx.encrypt(secret_key, twos_complement(10,size))
    w_x0=twos_complement(w_p[0],size)
    print("w_x1  is",w_x0)
    w_x1=twos_complement(w_p[1],size)

    w_x2=twos_complement(w_p[2],size)
    w_x3=twos_complement(w_p[3],size)
    w_x4=twos_complement(w_p[4],size)
    w_x5=twos_complement(w_p[5],size)
    print("5 and 6 is",w_p[5],w_p[6])
    w_x6=twos_complement(w_p[6],size)
    w_x7=twos_complement(w_p[7],size)
    w_x8=twos_complement(w_p[8],size)
    w_x9=twos_complement(w_p[9],size)
    w_x10=twos_complement(w_p[10],size)
    bias_x0=twos_complement(w_p[11],size)

    w_ciphertext1=ctx.encrypt(secret_key, w_x0)
    print("ciy:",ctx.decrypt(secret_key, w_ciphertext1))
    w_ciphertext2=ctx.encrypt(secret_key, w_x1)
    w_ciphertext3=ctx.encrypt(secret_key, w_x2)
    w_ciphertext4=ctx.encrypt(secret_key, w_x3)
    w_ciphertext5=ctx.encrypt(secret_key, w_x4)
    w_ciphertext6=ctx.encrypt(secret_key, w_x5)
    w_ciphertext7=ctx.encrypt(secret_key, w_x6)
    w_ciphertext8=ctx.encrypt(secret_key, w_x7)
    w_ciphertext9=ctx.encrypt(secret_key, w_x8)
    w_ciphertext10=ctx.encrypt(secret_key, w_x9)
    w_ciphertext11=ctx.encrypt(secret_key, w_x10)
    bias_ciphertext=ctx.encrypt(secret_key, bias_x0)
    start_time = time.time()
    for i in range(test_size):
        print(lines[i][0],lines[i][1],lines[i][2],lines[i][3],lines[i][4],lines[i][5],lines[i][6],lines[i][7],lines[i][8],lines[i][9],lines[i][10])
        temp=int(lines[i][0])
        b_x0=twos_complement(temp,size)
        temp=int(lines[i][1])
        b_x1=twos_complement(temp,size)
        temp=int(lines[i][2])
        b_x2=twos_complement(temp,size)
        temp=int(lines[i][3])
        b_x3=twos_complement(temp,size)
        temp=int(lines[i][4])
        b_x4=twos_complement(temp,size)
        temp=int(lines[i][5])
        b_x5=twos_complement(temp,size)
        temp=int(lines[i][6])
        b_x6=twos_complement(temp,size)
        temp=int(lines[i][7])
        b_x7=twos_complement(temp,size)
        temp=int(lines[i][8])
        b_x8=twos_complement(temp,size)
        temp=int(lines[i][9])
        b_x9=twos_complement(temp,size)
        temp=int(lines[i][10])
        b_x10=twos_complement(temp,size)
        ciphertext1=ctx.encrypt(secret_key, b_x0)
        ciphertext2=ctx.encrypt(secret_key, b_x1)
        ciphertext3=ctx.encrypt(secret_key, b_x2)
        ciphertext4=ctx.encrypt(secret_key, b_x3)
        ciphertext5=ctx.encrypt(secret_key, b_x4)
        ciphertext6=ctx.encrypt(secret_key, b_x5)
        ciphertext7=ctx.encrypt(secret_key, b_x6)
        ciphertext8=ctx.encrypt(secret_key, b_x7)
        ciphertext9=ctx.encrypt(secret_key, b_x8)
        ciphertext10=ctx.encrypt(secret_key, b_x9)
        ciphertext11=ctx.encrypt(secret_key, b_x10)
        presult_mul1 = mul(ciphertext1,w_ciphertext1,enc_temp)
        res=ctx.decrypt(secret_key, presult_mul1)
        val=listToString(res)
        print("partialpredict1mul ",two2dec(val[::-1]) )
        presult_mul2 = mul(ciphertext2,w_ciphertext2,enc_temp)
        res=ctx.decrypt(secret_key, presult_mul2)
        val=listToString(res)
        print("partialpredict2mul ",two2dec(val[::-1]) )
        presult_mul3 = mul(ciphertext3,w_ciphertext3,enc_temp)
        res=ctx.decrypt(secret_key, presult_mul3)
        val=listToString(res)
        print("partialpredict3mul ",two2dec(val[::-1]) )
        presult_mul4 = mul(ciphertext4,w_ciphertext4,enc_temp)
        res=ctx.decrypt(secret_key,presult_mul4)
        val=listToString(res)
        print("partialpredict4mul ",two2dec(val[::-1]) )
        presult_mul5 = mul(ciphertext5,w_ciphertext5,enc_temp)
        res=ctx.decrypt(secret_key, presult_mul5)
        val=listToString(res)
        print("partialpredict5mul ",two2dec(val[::-1]) )
        presult_mul6 = mul(ciphertext6,w_ciphertext6,enc_temp)
        res=ctx.decrypt(secret_key, presult_mul6)
        val=listToString(res)
        print("partialpredict6mul ",two2dec(val[::-1]) )
        presult_mul7 = mul(ciphertext7,w_ciphertext7,enc_temp)
        res=ctx.decrypt(secret_key, presult_mul7)
        val=listToString(res)
        print("partialpredict7mul ",two2dec(val[::-1]) )
        presult_mul8 = mul(ciphertext8,w_ciphertext8,enc_temp)
        res=ctx.decrypt(secret_key, presult_mul8)
        val=listToString(res)
        print("partialpredict8mul ",two2dec(val[::-1]) )
        presult_mul9 = mul(ciphertext9,w_ciphertext9,enc_temp)
        res=ctx.decrypt(secret_key, presult_mul9)
        val=listToString(res)
        print("partialpredict9mul ",two2dec(val[::-1]) )
        presult_mul10 = mul(ciphertext10,w_ciphertext10,enc_temp)
        res=ctx.decrypt(secret_key,presult_mul10)
        val=listToString(res)
        print("partialpredict10mul ",two2dec(val[::-1]) )
        presult_mul11 = mul(ciphertext11,w_ciphertext11,enc_temp)
        res=ctx.decrypt(secret_key, presult_mul11)
        val=listToString(res)
        print("partialpredict11mul ",two2dec(val[::-1]) )
        presult_add1 = add(presult_mul1, presult_mul2,enc_temp)
        res=ctx.decrypt(secret_key, presult_add1)
        val=listToString(res)
        print("partialpredictadd1 ",two2dec(val[::-1]) )
        presult_add2 = add(presult_mul3, presult_mul4,enc_temp)
        res=ctx.decrypt(secret_key, presult_add2)
        val=listToString(res)
        print("partialpredictadd2 ",two2dec(val[::-1]) )
        presult_add3 = add(presult_mul5, presult_mul6,enc_temp)
        res=ctx.decrypt(secret_key, presult_add3)
        val=listToString(res)
        print("partialpredictadd3 ",two2dec(val[::-1]) )
        presult_add4 = add(presult_mul7, presult_mul8,enc_temp)
        res=ctx.decrypt(secret_key, presult_add4)
        val=listToString(res)
        print("partialpredictadd4 ",two2dec(val[::-1]) )
        presult_add5 = add(presult_mul9, presult_mul10,enc_temp)
        res=ctx.decrypt(secret_key, presult_add5)
        val=listToString(res)
        print("partialpredictadd5 ",two2dec(val[::-1]) )
        #first step
        presult_add11 = add(presult_add1, presult_add2,enc_temp)
        presult_add12 = add(presult_add3, presult_add4,enc_temp)
        presult_add13 = add(presult_add5, presult_mul11,enc_temp)
        #secong step
        presult_add21 = add(presult_add11, presult_add12,enc_temp)
        wx= add(presult_add21, presult_add13,enc_temp)
        '''temp1=wx[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("wx",twos_comp_val(int(pa,2),len(pa)))'''
        signbit=wx[size-1]
        print("signbit is ",ctx.decrypt(secret_key, signbit))
        #print("wx type is",type(wx))
        #print("wx inside is",type(wx[0]))
        bits1 = [ctx.encrypt(secret_key,[False])] #[[vm.empty_ciphertext((1,))] for i in range(size)]
        #bits1=[[vm.empty_ciphertext((1,))] for i in range(1)]
        #print("bits1 datatype",type(bits1))
        #wxcpy=[[vm.empty_ciphertext((1,))] for i in range(size)]
        wxcpy=wx

        temp=wx
        wx=vm.gate_not(temp)


        wxtwos=add(wx,onep,enc_temp)
        x=mux(signbit,wxtwos,wxcpy)    
        m=add(x,onep,enc_temp)
        '''temp1=m
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("m values is",twos_comp_val(int(pa,2),len(pa)))'''    
        wxten=mul(wxcpy, eten, enc_temp)
        '''temp1=wxten[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("wxtenvalue",twos_comp_val(int(pa,2),len(pa)))'''
        sigmoid=nonRestoringDivision(m,wxten,size,enc_temp)
        '''temp1=sigmoid[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("divison result",twos_comp_val(int(pa,2),len(pa)))'''   
        bits1=sigmoid[i]
        sigmoidneg=vm.gate_not(bits1)
        sigmoidtwos=add(sigmoidneg,onep,enc_temp)
        '''temp1=sigmoidtwos[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("sigmoid2s",twos_comp_val(int(pa,2),len(pa))) '''   
        fsigmoid=mux(signbit,sigmoidtwos,sigmoid)      
        '''temp1=fsigmoid[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        #print("fsigmoid",pa)
        print("fsigmoid",twos_comp_val(int(pa,2),len(pa)))'''
        #print("fsigmoid",len(fsigmoid))
        '''temp1=presult_add3[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("thirdadd",pa)
        print("add3",twos_comp_val(int(pa,2),len(pa)))'''
        #partial_predict = addNumbers(presult_add3, bias,  size)
        #partial_predict.revesrse()
        '''result_bits = [ctx.decrypt(secret_key, partial_predict[i]) for i in range(size)]
        result_bits.reverse()
        pa=listToString(result_bits)
        print("partial predict",twos_comp_val(int(pa,2),len(pa)))'''

        predict1=predict(fsigmoid,onep,onen)
        res=ctx.decrypt(secret_key, predict1)
        val=listToString(res)
        print("2scomplement to dec ",two2dec(val[::-1]) )
        #print(" nuFHE multiplication number is : ", plain_predict[i])
        print("i value is",i, "endtime",datetime.now(),"start time",start_time)
        #print("start time is",start_time)
        #print("end time",time.time())
        print("endtime",datetime.now(),"start time",start_time,"execution time",time.time()-start_time)
        #result.reverse()
        #for j in range(test_size):
            #print(" nuFHE multiplication number is actual",lines[j][11],"preicted", plain_predict[j])

Please enter a string:
1
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1']
cix: [[ True]
 [ True]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]]
['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1']
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '0']
['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '0']
w_x1  is [[False], [False], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True]]
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0']
['1', '1', 

In [ ]:
import time
import random
#import pycuda.autoinit
import nufhe
import numpy as np
import numpy as np
from datetime import datetime


def threetotwo(r,p0,p1,p2):
  r[0] = vm.gate_or(p0, p1)
  temp=vm.gate_and(p0, p1)
  r[1]=vm.gate_or(temp, p2)
  return r
def twotoone(r, p0,p1 ):
  r[0] = vm.gate_or(p0, p1)
  return r
def is_equal(a,b,nBits):
    #print("type of a",type(a))
    #print("type of b",type(b))      
    t2= ctx.encrypt(secret_key, [False])
    #print("length of list",len(a))
    for i in range(0,nBits):
        t1=vm.gate_xor(a[i], b[i])
        t3=vm.gate_or(t2, t1)
        t2=vm.gate_copy(t3)
        equal=vm.gate_not(t3)

    return equal
def make_neg(n,nbits):
    list1=[int(i) for i in range(0,len(n)) ]
    listp=[]
    one= ctx.encrypt(secret_key, [True])
    zero= ctx.encrypt(secret_key, [False])
    t2= ctx.encrypt(secret_key, [False])
    #print(n)
    onep=  [ctx.encrypt(secret_key, [False]) for _ in range(nBits)]
    onep[0]=one 
    testone= [[vm.empty_ciphertext((1,))] for i in range(nBits)]
    testone=onep[:]
    '''temp1=n[:]
    temp1.reverse()
    result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(nBits)]
    pa=listToString(result_bits)
    print("n before",twos_comp_val(int(pa,2),len(pa)))'''
    n=addNumbers(n,testone,nBits)
    '''temp1=n[:]
    temp1.reverse()
    result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(nBits)]
    pa=listToString(result_bits)
    print("n after",twos_comp_val(int(pa,2),len(pa)))'''
    return n


def nonRestoringDivision(m, q,nBits ): 
    A=  [ctx.encrypt(secret_key, [False]) for _ in range(nBits)]
    temp=  [ctx.encrypt(secret_key, [False]) for _ in range(nBits)]
    mux1=  [ctx.encrypt(secret_key, [False]) for _ in range(nBits)]
    mux2=  [ctx.encrypt(secret_key, [False]) for _ in range(nBits)]
    neg2=  [ctx.encrypt(secret_key, [False]) for _ in range(nBits)]
    q_c=  [ctx.encrypt(secret_key, [False]) for _ in range(nBits)]
    mcopy=  [ctx.encrypt(secret_key, [False]) for _ in range(nBits)]
    onep=  [ctx.encrypt(secret_key, [False]) for _ in range(nBits)]
    fullzero=[ctx.encrypt(secret_key, [False]) for _ in range(nBits)]
    #one=[vm.empty_ciphertext((1,))]
    #zero=[vm.empty_ciphertext((1,))]
    one= ctx.encrypt(secret_key, [True])
    zero= ctx.encrypt(secret_key, [False])
    signbit= ctx.encrypt(secret_key, [False])
    #tw= ctx.encrypt(secret_key, [False])
    '''for i in range(len(q)):
        q_c=nufhe.LweSampleArray.copy(q[i])'''
    onep[0]=vm.gate_copy(one)  
    c_zero= [[vm.empty_ciphertext((1,))] for i in range(nBits)]
    testone= [[vm.empty_ciphertext((1,))] for i in range(nBits)]
    for i in range(nBits):
        c_zero[i]=vm.gate_copy(zero)
        #A[i]=vm.gate_copy(zero)
        testone[i]=vm.gate_copy(onep[i])
        q_c[i]=q[i]
        #A[i]=zero
    A=fullzero[:]
    mcopy=m[:]
    temp=c_zero[:]
    mux1=fullzero[:]
    equal_res= ctx.encrypt(secret_key, [False])
    neg2=make_neg(m,nBits)
    temp2=neg2[:]
    temp2.reverse()
    result_bits = [ctx.decrypt(secret_key, temp2[i]) for i in range(nBits)]
    pa=listToString(result_bits)
    print("neg2 value",twos_comp_val(int(pa,2),len(pa)))
    count=16
    print("********* loop started **************")
    while (count): 
        print("===loop==",count)
        singbit=vm.gate_copy(A[len(A)-1])
        signplain=[]
        signplain.append([ctx.decrypt(secret_key,signbit)])
        print("signbit is",signplain)
        for i in range(len(A)-1,0,-1):
            A[i] =nufhe.LweSampleArray.copy(A[i-1])    
        A[0]=vm.gate_copy(q_c[len(q_c)-1])
        for i in range(len(A)-1,0,-1):
            q_c[i] =vm.gate_copy(q_c[i-1])
        temp2=A[:]
        temp2.reverse()
        result_bits = [ctx.decrypt(secret_key, temp2[i]) for i in range(nBits)]
        pa=listToString(result_bits)
        print("A after left shitf",twos_comp_val(int(pa,2),len(pa)))
        #aminusm=subtract(A,m,nBits)
        #aplusm=addNumbers(A,m,nBits)
        for i in range(0,nBits):
            mux1[i]=vm.gate_mux(signbit,mcopy[i],neg2[i])
        temp2=mcopy[:]
        temp2.reverse()
        result_bits = [ctx.decrypt(secret_key, temp2[i]) for i in range(nBits)]
        pa=listToString(result_bits)
        print("m before add",twos_comp_val(int(pa,2),len(pa)))        
        A=addNumbers(A,mux1,nBits)
        temp2=A[:]
        temp2.reverse()
        result_bits = [ctx.decrypt(secret_key, temp2[i]) for i in range(nBits)]
        pa=listToString(result_bits)
        print("A value loop",twos_comp_val(int(pa,2),len(pa)))
        #A=mux1[:]
        #equal_res=is_equal(A,c_zero,nBits)
        signbit=vm.gate_copy(A[len(A)-1])
        signplain1=[]
        signplain1.append([ctx.decrypt(secret_key,signbit)])
        #print("@@@ signbit beforeupdate of q_C",signplain1)
        tw1=vm.gate_mux(signbit,zero,one)
        signplain2=[]
        signplain2.append([ctx.decrypt(secret_key,tw1)])
        #print("@@@ tw1 beforeupdate of q_C",signplain2)
        q_c[0]=vm.gate_copy(tw1)
        #q_c.insert(0,tw1)
        #temp2 = [[vm.empty_ciphertext((1,))] for i in range(nBits)]
        temp2=A[:]
        temp2.reverse()
        result_bits = [ctx.decrypt(secret_key, temp2[i]) for i in range(nBits)]
        pa=listToString(result_bits)
        print("count is" ,count,"A inside loop",twos_comp_val(int(pa,2),len(pa)))
        temp2=q_c[:]
        temp2.reverse()
        result_bits = [ctx.decrypt(secret_key, temp2[i]) for i in range(nBits)]
        pa=listToString(result_bits)
        print("count is" ,count,"q_c inside loop",twos_comp_val(int(pa,2),len(pa)))
        count -= 1
    #aminuszero=subtract(A,c_zero,nBits)
    signbit=vm.gate_copy(A[len(A)-1])
    aminuszero=A[:]
    aplusm =addNumbers(A,c1,nBits)
    for i in range(0,nBits):
        A[i]=vm.gate_mux(signbit,aplusm[i],aminuszero[i])    
    return q_c
    #return A,q_c 


def fixSizeBoolList(decimal,size):
    x = [int(x) for x in bin(decimal)[2:]]
    x = list(map(bool, x))
    x = [False]*(size - len(x)) + x
    pow2 = []
    for i in range(size):
      pow2.append([x[i]])
    pow2.reverse()
    return pow2
def boolListToInt(bitlists):
    out = 0
    for bit in bitlists:
        out = (out << 1) | bit
    return out

def addBits(r, a, b, carry):
    # Xor(t1[0], a, carry[0])
    t1 = vm.gate_xor(a, b)
    # Xor(t2[0], b, carry[0])
    # Xor(r[0], a, t2[0])
    r[0] = vm.gate_xor(t1, carry)
    # And(t1[0], t1[0], t2[0])
    t2 = vm.gate_and(a, carry)
    t3 = vm.gate_and(b, carry)
    t4=vm.gate_and(a,b)
    t5= vm.gate_or(t2,t3)
    # Xor(r[1], carry[0], t1[0])
    r[1] = vm.gate_or(t5, t4)

    return r
 

def addNumbers(ctA, ctB, nBits):
    ctRes = [[vm.empty_ciphertext((1,))] for i in range(nBits)]
    # carry = vm.empty_ciphertext((1,))
    bitResult = [[vm.empty_ciphertext((1,))] for i in range(2)]
    ctRes[0] = vm.gate_xor(ctA[0], ctB[0])
    # Xor(ctRes[0], ctA[0], ctB[0])
    carry = vm.gate_and(ctA[0], ctB[0])
    # And(carry[0], ctA[0], ctB[0])
    for i in range(1,nBits ):
      if i>4:
        bitResult = addBits(bitResult, ctA[i], ctB[i], carry)
        # Copy(ctRes[i], bitResult[0]);
        ctRes[i] = nufhe.LweSampleArray.copy(bitResult[0])
        # Copy(carry[0], bitResult[1])
        carry = nufhe.LweSampleArray.copy(bitResult[1])
    else:
        bitResult = twotoone(bitResult, ctA[i], ctB[i])
        #bitResult = threetotwo(bitResult, ctA[i], ctB[i],carry)
        # Copy(ctRes[i], bitResult[0]);
        ctRes[i] = nufhe.LweSampleArray.copy(bitResult[0])
        #carry = nufhe.LweSampleArray.copy(bitResult[1])
        

    return ctRes


def mulNumbers(ctA, ctB, secret_key, input_bits, output_bits):
    result = [ctx.encrypt(secret_key, [False]) for _ in
              range(output_bits)]
    temp = [ctx.encrypt(secret_key, [False]) for _ in
              range(output_bits)]          
    # [[vm.empty_ciphertext((1,))] for _ in range(output_bits)]
    # andRes = [[vm.empty_ciphertext((1,))] for _ in range(input_bits)]
    
    for i in range(input_bits):
        andResLeft = [ctx.encrypt(secret_key, [False]) for _ in
                      range(output_bits)]
        #temp=mux(temp,ctA,ctB[i],size)             
        for j in range(input_bits):
            andResLeft[j + i] = vm.gate_and(ctA[j], ctB[i])

            # andResLeft[j + i] = nufhe.LweSampleArray.copy(andRes[j])
        result = addNumbers(andResLeft, result, output_bits)
        #result_bits = [ctx.decrypt(secret_key, result[i]) for i in range(size * 2)]
        #print(" nuFHE multiplication intermdiate number is : ",i,boolListToInt(result_bits))

    return result[:size]
def Convert_list(string):
    list1=[]
    list1[:0]=string
    print(list1)
    list1=[int(i)for i in list1 ]
    listb=[]
    for i in list1:
        if i==0:
            listb.append([False])
        else:
            listb.append([True])    
    
    #print(listb)
    return listb

def twos_complement(n,nbits):
    a=f"{n & ((1 << nbits) - 1):0{nbits}b}" 
    #print(type(a))
    a=Convert_list(a)
    a.reverse()
    return a
def listToString(s): 
    # initialize an empty string
    list1=[int(i)for i in s ]
    listp=[]
    for i in list1:
        if i==False:
            listp.append('0')
        else:
            listp.append('1')    
    
    #print(listp)
    str1 = "" 
    # traverse in the string  
    s=['delim'.join([str(elem) for elem in sublist]) for sublist in listp]
    #print(s)
    for ele in s: 
        str1 += ele  
    # return string 
    #print(str1)
    return str1
def twos_comp_val(val,bits):
    """compute the 2's complement of int value val"""
    #val=listToString(val)
    

    if (val & (1 << bits - 1)) != 0: # if sign bit is set e.g., 8bit: 128-255
        val = val - (1 << bits)        # compute negative value
    return val
def compare_bit(  a, b,  lsb_carry,  tmp): 
  result= ctx.encrypt(secret_key, [False])
  tmp=vm.gate_xnor(a, b)
  result=vm.gate_mux(tmp,lsb_carry, a)
  return result
def minimum(  a,  b,  nb_bits): 
    tmps1= ctx.encrypt(secret_key, [False])
    tmps2= ctx.encrypt(secret_key, [True])
    #initialize the carry to 0
    #run the elementary comparator gate n times
    for i in range(nb_bits):
      tmps1= compare_bit(a[i],b[i],tmps1,tmps2)
    #tmps[0] is the result of the comparaison: 0 if a is larger, 1 if b is larger
    #select the max and copy it to the result
    return tmps1
def predict(ctA,secret_key, output_bits):
    zero = [ctx.encrypt(secret_key, [False]) for _ in range(output_bits)]
    onen=  [ctx.encrypt(secret_key, [True]) for _ in range(output_bits)]        
    onep=  [ctx.encrypt(secret_key, [False]) for _ in range(output_bits)]
    one= ctx.encrypt(secret_key, [True])
    temp= ctx.encrypt(secret_key, [True])
    temp=ctA[output_bits-1]
    comp_res= ctx.encrypt(secret_key, [True])
    onep[0]=one
    ctRes = [ctx.encrypt(secret_key, [False]) for _ in range(output_bits)]
    # Copy(ctRes[i], bitResult[0]);
    #comp_res= minimum(ctA,zero,output_bits)
    #temp=
    #comp_res=
    for i in range(output_bits):
      ctRes[i] = vm.gate_mux(temp,onen[i],onep[i])  
    # Copy(carry[0], bitResult[1])
    return ctRes
def predict(ctA,onep,onen):
    c_s=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    for i in range(size):
        c_s[i]=ctA[size-1]   
    res= vm.gate_mux(c_s,onen,onep)
def mul(ciX,ciY,enc_temp):
    
    msb_temp= [vm.empty_ciphertext((1,))]
    control=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    c_s=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    c_s1=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    msb_temp=vm.gate_xor(ciX[size-1],ciY[size-1])
    t=size-1
    for i in range(size):
            control[i]=ciX[t]
            c_s1[i]=ciY[t]
    mul_temp=ciX
    zero=enc_temp
    one_ciX=vm.gate_not(ciX)
    result=ctx.decrypt(secret_key, one_ciX)
    one_ciY=vm.gate_not(ciY)
    comp_ciX=add(one_ciX,one)
    comp_ciY=add(one_ciY,one)
    
    #print("control bit:",ctx.decrypt(secret_key, control))
    ciX=mux(control,comp_ciX,ciX)
    ciY=mux(c_s1,comp_ciY,ciY)
    #print("cix:",ctx.decrypt(secret_key, ciX))
    for j in range(0,size):
        #print("=================iteration==========",j)
        
        
        #print("cix:",ctx.decrypt(secret_key, ciX))
        for i in range(size):
            c_s[i]=ciY[j]
        add_res=mux(c_s,ciX,zero)
        
        enc_theta=add(enc_temp,add_res)
        enc_temp=enc_theta
        #enc_temp=ciX
        #print("enc_teta:",ctx.decrypt(secret_key, enc_theta))
        
        #print("control bit:",ctx.decrypt(secret_key, c_s[0]))
        #mux_result=mux(c_s,enc_theta,ciX)
        #mux_result=vm.gate_mux(c_s, enc_theta,ciX)
        #print("mux result:",ctx.decrypt(secret_key, mux_result))
        #ciX=mux_result
        ciX.roll(1, axis=0)
        #ciX.roll(1, axis=0)
    for i in range(size):
        c_s[i]=msb_temp    
    one_presult=vm.gate_not(enc_theta)
    comp_presult=add(one_presult,one)
    ciX=vm.gate_mux(c_s,comp_presult,enc_theta)
    return ciX
def mux(c_s,true,false) :
    temp1=vm.gate_and(c_s,true)
    #print("temp1:",ctx.decrypt(secret_key, temp1))
    temp2=vm.gate_and(vm.gate_not(c_s),false)
    #print("temp2:",ctx.decrypt(secret_key, temp2))
    mux_result=vm.gate_or(temp1,temp2)
    return mux_result
def two2dec(s):
    if s[0] == '1':
        return -1 * (int(''.join('1' if x == '0' else '0' for x in s), 2) + 1)
    else:
        return int(s, 2)
if __name__ == '__main__':    
    ctx = nufhe.Context()
    secret_key, cloud_key = ctx.make_key_pair()
    vm = ctx.make_virtual_machine(cloud_key)
    size=16
    onep=  [ctx.encrypt(secret_key, [False]) for _ in range(size)]
    one= ctx.encrypt(secret_key, [True])
    onep[0]=one
    with open('/content/drive/MyDrive/train.txt') as f:
      lines = []
      for line in f:
        lines.append([int(v) for v in line.split()])
    
    bits = [[False] for i in range(size - 2)]
    zeros = [[False] for i in range(size)]

    test_size = int(input("Please enter a string:\n"))
    '''b_x0=[]
    b_x1=[]
    b_x2=[]
    b_x3=[]
    for i in range(test_size):
        temp=int(lines[i][0])
        #print(type(temp))
        b_x0.append(fixSizeBoolList(temp,size))
        temp=int(lines[i][1])
        b_x1.append(fixSizeBoolList(temp,size))
        temp=int(lines[i][2])
        b_x2.append(fixSizeBoolList(temp,size))
        temp=int(lines[i][3])
        b_x3.append(fixSizeBoolList(temp,size))'''
    #print(b_x0,b_x1,b_x2,b_x3)
    #-3.5133618, -4.2616279 , 10.59763759,  6.17913037, -0.1758181
    w_p=[4,-8 ,22 ,9 , -1]
    #w_p=[-3, -4 , 10,  6, -1]
    
    #print(type(x[0]))
    #print(x)
    #b_y = fixSizeBoolList(deci_y,size)
    #print(type(y[0]))
    #print(y)
    #x.reverse()
    #print(x)
    #y.reverse()
    #print(y)
    featuresize=5
    w=[]
    w_b=[]
    for i in range(featuresize-1):
        w.append([[vm.empty_ciphertext((1,))] for i in range(size)])
        w_b.append(twos_complement(w_p[i],size))
        for j in range(size):
            w[i][j] = ctx.encrypt(secret_key, w_b[i][j])
    bias=[[vm.empty_ciphertext((1,))] for i in range(size)]
    bias_b=twos_complement(w_p[4],size)
    eten=[[vm.empty_ciphertext((1,))] for i in range(size)]
    eten_b=twos_complement(10,size)
    for i in range(size):
      bias[i]=ctx.encrypt(secret_key, bias_b[i])
      eten[i]=ctx.encrypt(secret_key, eten_b[i])
    
    plain_predict=[]
    start_time = datetime.now()
    for i in range(test_size):
        print(lines[i][0],lines[i][1],lines[i][2],lines[i][3])
        temp=int(lines[i][0])
        #print(type(temp))
        #x = twos_complement(deci_x,size)
        b_x0=twos_complement(temp,size)
        temp=int(lines[i][1])
        b_x1=twos_complement(temp,size)
        temp=int(lines[i][2])
        b_x2=twos_complement(temp,size)
        temp=int(lines[i][3])
        b_x3=twos_complement(temp,size)
        ciphertext1=[[vm.empty_ciphertext((1,))] for i in range(size)]
        ciphertext2=[[vm.empty_ciphertext((1,))] for i in range(size)]
        ciphertext3=[[vm.empty_ciphertext((1,))] for i in range(size)]
        ciphertext4=[[vm.empty_ciphertext((1,))] for i in range(size)]
        for j in range(size):
            ciphertext1[j] = ctx.encrypt(secret_key, b_x0[j])
            ciphertext2[j] = ctx.encrypt(secret_key, b_x1[j])
            ciphertext3[j] = ctx.encrypt(secret_key, b_x2[j])
            ciphertext4[j] = ctx.encrypt(secret_key, b_x3[j])
        #ciphertext1 = ctx.encrypt(secret_key, x)
        #ciphertext2 = ctx.encrypt(secret_key, y) 
        start_time = time.time()
        #ciphertext1.reverse()
        #ciphertext2.reverse()
        #result = addNumbers(ciphertext1, ciphertext2, size)
        #partial_predict=[]

        #print(type(ciphertext1[0][0]))
        temp1 = [[vm.empty_ciphertext((1,))] for i in range(size)]
        temp2 = [[vm.empty_ciphertext((1,))] for i in range(size)]
        #predict[i]=[[vm.empty_ciphertext((1,))] for i in range(size*2)]
        #temp1=[]
        #temp2=[]
        presult_mul1 = mulNumbers(ciphertext1,w[0], secret_key, size, size * 2)
        '''temp1=presult_mul1[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("mul1",twos_comp_val(int(pa,2),len(pa)))'''
        presult_mul2 = mulNumbers(ciphertext2, w[1], secret_key, size, size * 2)
        '''temp1=presult_mul2[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("mul2",twos_comp_val(int(pa,2),len(pa)))'''
        presult_mul3 = mulNumbers(ciphertext3, w[2], secret_key, size, size * 2)
        '''temp1=presult_mul3[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("mul3",twos_comp_val(int(pa,2),len(pa)))'''
        presult_mul4 = mulNumbers(ciphertext4, w[3], secret_key, size, size * 2)
        '''temp1=presult_mul4[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("mul4",twos_comp_val(int(pa,2),len(pa)))'''
        presult_add1 = addNumbers(presult_mul1, presult_mul2,  size)
        '''temp1=presult_add1[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("firstadd",pa)
        print("add1",twos_comp_val(int(pa,2),len(pa)))'''
        presult_add2 = addNumbers(presult_mul3, presult_mul4,  size)
        '''temp1=presult_add2[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("secondadd",pa)
        print("add2",twos_comp_val(int(pa,2),len(pa)))'''

        presult_add2= addNumbers(presult_add1, presult_add2,  size)
        wx=presult_add2[:]
        signbit=vm.gate_copy(wx[len(wx)-1])
        for i in range(size):
            #temp=wx[i]
            wx[i]=vm.gate_not(wx[i])
            #wx[i]=temp
        wxtwos=make_neg(wx,size)
        
        for i in range(len(wx)):
          x[i]=vm.vm.gate_mux(signbit,wxtwos[i],wx[i])
        m=addNumbers(x,onep,size)
        wxten=mulNumbers(x, eten, secret_key, size, size * 2)
        
        sigmoid=nonRestoringDivision(m,wxten,size)
        
        
        '''temp1=presult_add3[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("thirdadd",pa)
        print("add3",twos_comp_val(int(pa,2),len(pa)))'''
        #partial_predict = addNumbers(presult_add3, bias,  size)
        #partial_predict.revesrse()
        '''result_bits = [ctx.decrypt(secret_key, partial_predict[i]) for i in range(size)]
        result_bits.reverse()
        pa=listToString(result_bits)
        print("partial predict",twos_comp_val(int(pa,2),len(pa)))'''
        predict1=predict(sigmoid,secret_key,size)
        predict1.reverse()
        #print(predict1)
        #print(type(predict1))
        #print(type(predict1[0]))
        #result_bits = [ctx.decrypt(secret_key, predict1[i]) for i in range(size*2)]
        result_bits = [ctx.decrypt(secret_key, predict1[i]) for i in range(size)]
        #plain_predict.append(boolListToInt(result_bits))
        pa=listToString(result_bits)
        plain_predict.append(twos_comp_val(int(pa,2),len(pa)))
        print(" nuFHE multiplication number is : ", plain_predict[i])
    #print("start time is",start_time)
    #print("end time",time.time())
    print("endtime",datetime.now(),"start time",start_time)
    #result.reverse()
    for j in range(test_size):
        print(" nuFHE multiplication number is actual",lines[j][4],"preicted", plain_predict[j])


Please enter a string:
20
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0']
['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '0', '0']
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '1', '0']
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '1']
['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '0']
51 25 30 11
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0', '1', '1']
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0', '1']
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '1', '1', '0']
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '1']


AttributeError: ignored

time: 6min 1s (started: 2021-12-17 08:51:57 +00:00)
